In [21]:
import os
import pandas as pd

In [22]:
# Open up csv table with info
data_csv = '/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data.csv'
df = pd.read_csv(data_csv)

# Delete not usefull columns
df = df.drop(columns=['Age', 'Sex', 'Height', 'Weight', 'Pregnancy status', 'Most audible location', 'Systolic murmur pitch', 'Systolic murmur quality', 'Diastolic murmur timing', 'Diastolic murmur shape', 'Diastolic murmur grading', 'Diastolic murmur pitch', 'Diastolic murmur quality', 'Campaign', 'Additional ID', 'Systolic murmur grading', 'Systolic murmur shape', 'Systolic murmur timing'])

In [23]:
os.mkdir('/kaggle/working/training_data')
ofilepathi = r'/kaggle/working/training_data' 

FileExistsError: [Errno 17] File exists: '/kaggle/working/training_data'

In [ ]:
# For each patient make unique row for each recording location, add digit to end of patient ID corresponding to recording location
new_df = pd.DataFrame(columns=['Patient ID', 'Label', 'Recording location', 'filepath', 'ofilepath'])
for index, row in df.iterrows():
  try:
    recording_locations = row['Locations'].split('+')
  except:
    print("no recording location")
  patient_id = row['Patient ID']
  murmur = row['Murmur']

  # Turning Nan data cells into variable of empty list
  if type(row['Murmur locations']) != float:
    murmur_locations = row['Murmur locations'].split('+')
  else:
    murmur_locations = list()

  # Iterate over recording locations and add new row
  for recording_loc in recording_locations:
      # Get filepath
      filepath = '/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/' + str(patient_id) + "_" + recording_loc + '.wav'
      ofilepath = ofilepathi+ '/' + str(patient_id) + "_" + recording_loc + '.wav'
      # Go through and get number  corresponding to location: {"AV" : 1, "PV" : 2, "TV" : 3, "MV" : 4, "Phc(any location) : 5"}
      if recording_loc == 'AV':
        integer_add = 1
      elif recording_loc == 'PV':
        integer_add = 2
      elif recording_loc == 'TV':
        integer_add = 3
      elif recording_loc == 'MV':
        integer_add = 4
      else:
        integer_add = 5

      # Checking if recording location had murmur
      if recording_loc in murmur_locations:      
        new_row = {'Patient ID' : patient_id, 'Label' : 1, 'Recording location' : integer_add, 'filepath' : filepath, 'ofilepath' : ofilepath}
        new_df = new_df.append(new_row, ignore_index = True)
      else:
        # Check to see if murmur is unkown or absent
        if murmur == "Unknown":
          new_row = {'Patient ID' : patient_id, 'Label' : 0, 'Recording location' : integer_add, 'filepath' : filepath, 'ofilepath' : ofilepath}
          new_df = new_df.append(new_row, ignore_index = True)
        else:
          new_row = {'Patient ID' : patient_id, 'Label' : 0, 'Recording location' : integer_add, 'filepath' : filepath, 'ofilepath' : ofilepath}
          new_df = new_df.append(new_row, ignore_index = True)

In [ ]:
# Labels: {0: Not there/unknown, 1 : there}, {0 : 'Absent', 1 : 'AV', 2 : 'PV', 3 : 'TV', 4 : 'MV', 0 : 'unknown'}
new_df = new_df.drop(columns=['Recording location'])
new_df.head(300)

In [ ]:
# Delete duplicates
new_df = new_df.drop_duplicates()

##Splice Audio files to only 10 heart cycle annotated segments

In [24]:
from os.path import exists
from pydub import AudioSegment

for index, row in new_df.iterrows():
  try:
    filepath = row['filepath']
    ofilepath = row['ofilepath']
    # Check if file path exists
    if exists(filepath):
      # Get TSV file
      tsv_filepath = filepath.replace('.wav', '.tsv')
      tsv = pd.read_csv(tsv_filepath, sep='\t', names=['start', 'end', 'label'])

      # Go through rows and cut .wav file to only 10 annotated segments
      # Define ten_time to increase every time there is a beggining of a heart beat(i.e. label==1)
      ten_time = 0
      no_start = True
      for tsv_index, tsv_row in tsv.iterrows():
        # First 1 to start clip
        if tsv_row['label'] == 1 and no_start == True:
          start = tsv_row['start']
          ten_time+=1
          no_start = False
          continue
        # Update ten_time without assigning start to new later number
        elif tsv_row['label'] == 1:
          ten_time+=1
          continue
        # Annotated heart cycle repeated at least 10 times and label 4 was detected
        elif tsv_row['label'] == 4 and ten_time >= 10:
          end = tsv_row['end']
          break
        # Making sure last 4 annotation is up to date incase there are less than 10 cycles
        elif tsv_row['label'] == 4:
          end = tsv_row['end']
          continue

      # Either loop broke by hitting label 4 after 10 cycles or the loop just ended because there were under 10 4s
      start = start*1000
      end = end*1000

      # Get wav file and cut output not annotated part
      wav_file = AudioSegment.from_file(filepath)

      new_wav_file = wav_file[start:end]
      #print(filepath)
      new_wav_file.export(ofilepath, format='wav')
    else:
      # Audio has two files, have to do cropping for both
      filepath_mults = [filepath.split(".wav")[0] + "_1.wav", filepath.split(".wav")[0] + "_2.wav"]
      for filepath in filepath_mults:
        # Get TSV file
        tsv_filepath = filepath.replace('.wav', '.tsv')

        tsv = pd.read_csv(tsv_filepath, sep='\t', names=['start', 'end', 'label'])

        # Go through rows and cut .wav file to only 10 annotated segments
        # Define ten_time to increase every time there is a beggining of a heart beat(i.e. label==1)
        ten_time = 0
        no_start = True
        for tsv_index, tsv_row in tsv.iterrows():
          # First 1 to start clip
          if tsv_row['label'] == 1 and no_start == True:
            start = tsv_row['start']
            ten_time+=1
            no_start = False
            continue
          # Update ten_time without assigning start to new later number
          elif tsv_row['label'] == 1:
            ten_time+=1
            continue
          # Annotated heart cycle repeated at least 10 times and label 4 was detected
          elif tsv_row['label'] == 4 and ten_time >= 10:
            end = tsv_row['end']
            break
          # Making sure last 4 annotation is up to date incase there are less than 10 cycles
          elif tsv_row['label'] == 4:
            end = tsv_row['end']
            continue

        # Either loop broke by hitting label 4 after 10 cycles or the loop just ended because there were under 10 4s
        start = start*1000
        end = end*1000

        print(filepath)
        print(start)
        print(end)
        # Get wav file and cut output not annotated part
        wav_file = AudioSegment.from_file(filepath)

        new_wav_file = wav_file[start:end]
        print(len(new_wav_file))
        new_wav_file.export(ofilepath, format='wav')
  except Exception as e:
    print("Except", e)
    continue

  #break

/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/49748_AV_1.wav
8320.25
16520.25
8200
/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/49748_AV_2.wav
11300.25
19760.25
8460
/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/49748_PV_1.wav
740.25
8920.25
8180
/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/49748_PV_2.wav
580.25
9000.25
8420
/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/49952_MV_1.wav
1000.3539999999999
7220.354
6220
/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/49952_MV_2.wav
9520.25
14400.25
4880
/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/50032_TV_1.wav
1440.25
4900.25
3460
/kaggle/input/the-circor-digiscope-phonocardiogram-dataset-v2/training_data/training_data/50032_TV_2.wav

Delete all rows that have an audio file length of 0

In [25]:
pth = r'/kaggle/working/training_data'
print(len(os.listdir(pth)))

3140


In [26]:
for index, row in new_df.iterrows():
  filepath = row['ofilepath']
  if not exists(filepath):
        new_df = new_df.drop(index)
        print(index)
        continue
  if exists(filepath):
      wav_file = AudioSegment.from_file(filepath)
      if len(wav_file) == 0:
        new_df = new_df.drop(index)
        print(row)
  else:
    # Audio has two files, have to do cropping for both
    filepath_mults = [filepath.split(".wav")[0] + "_1.wav", filepath.split(".wav")[0] + "_2.wav"]
    for filepath in filepath_mults:
      wav_file = AudioSegment.from_file(filepath)
      if len(wav_file) == 0:
        new_df = new_df.drop(index)
        print(row)
    

Create Classifier Model Dataset


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import librosa

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm

In [28]:
# Split dataframe into train and val
from sklearn.model_selection import train_test_split

train, valid = train_test_split(new_df, test_size=0.2)
train, test = train_test_split(train, test_size=0.1)

In [29]:
valid.head()

,Patient ID,Label,filepath,ofilepath
1894,84689,0,/kaggle/input/the-circor-digiscope-phonocardio...,/kaggle/working/training_data/84689_PV.wav
1240,50771,0,/kaggle/input/the-circor-digiscope-phonocardio...,/kaggle/working/training_data/50771_MV.wav
2981,85249,0,/kaggle/input/the-circor-digiscope-phonocardio...,/kaggle/working/training_data/85249_TV.wav
1094,50699,0,/kaggle/input/the-circor-digiscope-phonocardio...,/kaggle/working/training_data/50699_TV.wav
1220,50762,0,/kaggle/input/the-circor-digiscope-phonocardio...,/kaggle/working/training_data/50762_AV.wav


In [30]:
# Get longest audio file
'''longest_duration = 0
longest_path = ''
for index, row in new_df.iterrows():
  file_path = row['filepath']
  try:
    wav,sr = librosa.load(file_path,sr=None)
    duration = librosa.get_duration(y=wav, sr=sr)
  except:
    # Combine two files
    print("combine")
    combine_sound(file_path)
    wav,sr = librosa.load(file_path,sr=None)
    duration = librosa.get_duration(y=wav, sr=sr)
  if duration > longest_duration:
    longest_duration = duration
    longest_path = file_path
print(longest_duration, longest_path)'''

'longest_duration = 0\nlongest_path = \'\'\nfor index, row in new_df.iterrows():\n  file_path = row[\'filepath\']\n  try:\n    wav,sr = librosa.load(file_path,sr=None)\n    duration = librosa.get_duration(y=wav, sr=sr)\n  except:\n    # Combine two files\n    print("combine")\n    combine_sound(file_path)\n    wav,sr = librosa.load(file_path,sr=None)\n    duration = librosa.get_duration(y=wav, sr=sr)\n  if duration > longest_duration:\n    longest_duration = duration\n    longest_path = file_path\nprint(longest_duration, longest_path)'

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [32]:
def spec_to_image(spec, eps=1e-6):
  mean = spec.mean()
  std = spec.std()
  spec_norm = (spec - mean) / (std + eps)
  spec_min, spec_max = spec_norm.min(), spec_norm.max()
  spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
  spec_scaled = spec_scaled.astype(np.uint8)
  return spec_scaled

In [33]:
import wave

def combine_sound(file_path):
  infiles = [file_path.split(".wav")[0] + "_1.wav", file_path.split(".wav")[0] + "_2.wav"]

  outfile = file_path

  data= []
  for infile in infiles:
      w = wave.open(infile, 'rb')
      data.append( [w.getparams(), w.readframes(w.getnframes())] )
      w.close()
      
  output = wave.open(outfile, 'wb')
  output.setparams(data[0][0])
  for i in range(len(data)):
      output.writeframes(data[i][1])
  output.close()

In [34]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
#n_fft = 2048, n_mels = 128, fmax=8300, max_sound_time = 18
def get_melspectrogram_db(file_path, sr=None, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
  try:
    wav,sr = librosa.load(file_path,sr=sr)
  except:
    # Combine two files
    print("except")
    combine_sound(file_path)
    wav,sr = librosa.load(file_path,sr=sr)
  if wav.shape[0]<18*sr:
    wav=np.pad(wav,int(np.ceil((18*sr-wav.shape[0])/2)),mode='reflect')
  else:
    wav=wav[:18*sr]
  spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,
              hop_length=hop_length,n_mels=n_mels,fmin=fmin,fmax=fmax)
  spec_db=librosa.power_to_db(spec,top_db=top_db)
  #librosa.display.specshow(spec_db,sr=sr, x_axis='linear')
  #plt.show()
  return spec_db

In [35]:
class HeartMurmurData(Dataset):
  def __init__(self, df, in_col, out_col):
    self.df = df
    self.data = []
    self.labels = []
    self.c2i={}
    self.i2c={}
    self.categories = sorted(df[out_col].unique())
    for i, category in enumerate(self.categories):
      self.c2i[category]=i
      self.i2c[i]=category
    for ind in tqdm(range(len(df))):
      row = df.iloc[ind]
      file_path = row[in_col]
      self.data.append(spec_to_image(get_melspectrogram_db(file_path))[np.newaxis,...])
      self.labels.append(self.c2i[row['Label']])
  def __len__(self):
    return len(self.data)
  def __getitem__(self, idx):
    return self.data[idx], self.labels[idx]

In [ ]:
train_data = HeartMurmurData(train, 'ofilepath', 'Label')
valid_data = HeartMurmurData(valid, 'ofilepath', 'Label')

  0%|          | 0/2260 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.0007019  -0.00210571 -0.00048828 ...  0.01617432  0.02514648
  0.03115845] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/opt/conda/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass y=[0.01605225 0.00387573 0.00369263 ... 0.01696777 0.0161438  0.00909424] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.01535034  0.01815796  0.01623535 ... -0.01107788 -0.01004028
 -0.01473999] as keywo

Creating Classifier Model

In [ ]:
from torchvision.models import resnet152, mobilenet_v3_small
import torch
import torch.nn as nn
import torch.optim as optim
if torch.cuda.is_available():
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
resnet_model = mobilenet_v3_small(pretrained=True)
resnet_model.fc = nn.Linear(2048,2)
resnet_model.conv1 = nn.Conv2d(1, 3, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
resnet_model = resnet_model.to(device)

In [ ]:
class CustomMobileNetV2(nn.Module):
    def __init__(self):
        super(CustomMobileNetV2, self).__init__()
        # Add a Conv2d layer to convert 1 channel to 3 channels
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1)
        
        # Load the pretrained MobileNetV2 model (excluding the classifier)
        self.mobilenet = mobilenet_v3_small(pretrained=True)
        self.classifier = nn.Linear(1000,2)
        # Remove the classifier layer (fully connected layers)
        #self.mobilenet.classifier = nn.Identity()

    def forward(self, x):
        # Apply the Conv2d layer
        x = self.conv1(x)
        
        # Pass the output through the pretrained MobileNetV2 model
        x = self.mobilenet(x)
        x = self.classifier(x)
        return x

# Create an instance of the custom model
custom_mobilenet = CustomMobileNetV2()


In [ ]:
!pip install torchsummary
from torchsummary import summary

In [ ]:
summary(custom_mobilenet.to(device), (1, 128, 141))

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=8, shuffle=True)

In [ ]:
torch.cuda.empty_cache()

Training Classifier Model

In [ ]:
x, y=next(train_loader._get_iterator())
x.shape

In [ ]:
custom_mobilenet(x.to(device, dtype=torch.float32))

In [ ]:
plt.plot(np.concatenate(resnet_train_losses))

In [ ]:

learning_rate = 2e-5
optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)
epochs = 100
loss_fn = nn.CrossEntropyLoss()
resnet_train_losses=[]
resnet_valid_losses=[]
def lr_decay(optimizer, epoch):
  if epoch%10==0:
    new_lr = learning_rate / (2**(epoch//10))
    optimizer.param_groups[0]['lr'] = new_lr
    #optimizer = setlr(optimizer, new_lr)
    print(f'Changed learning rate to {new_lr}')
  return optimizer
def train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, train_losses, valid_losses, change_lr=None):
  for epoch in tqdm(range(1,epochs+1)):
    model.train()
    batch_losses=[]
    if change_lr:
      optimizer = change_lr(optimizer, epoch)
    for i, data in enumerate(train_loader):
      x, y = data
      optimizer.zero_grad()
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      y_hat = model(x)
      loss = loss_fn(y_hat, y)
      loss.backward()
      batch_losses.append(loss.item())
      optimizer.step()
    train_losses.append(batch_losses)
    print(f'Epoch - {epoch} Train-Loss : {np.mean(train_losses[-1])}')
    model.eval()
    batch_losses=[]
    trace_y = []
    trace_yhat = []
    for i, data in enumerate(valid_loader):
      x, y = data
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      y_hat = model(x)
      loss = loss_fn(y_hat, y)
      trace_y.append(y.cpu().detach().numpy())
      trace_yhat.append(y_hat.cpu().detach().numpy())      
      batch_losses.append(loss.item())
    valid_losses.append(batch_losses)
    trace_y = np.concatenate(trace_y)
    trace_yhat = np.concatenate(trace_yhat)
    accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)
    print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])} Valid-Accuracy : {accuracy}')
train(custom_mobilenet, loss_fn, train_loader, valid_loader, epochs, optimizer, resnet_train_losses, resnet_valid_losses, lr_decay)

Test Model

In [ ]:
test_data = HeartMurmurData(test, 'ofilepath', 'Label')

In [ ]:
test_loader = DataLoader(test_data, batch_size=8, shuffle=True)

In [ ]:
correct = 0
predictions_ar = []
labels_ar = []


custom_mobilenet.eval()
for i, (data, labels) in enumerate(test_loader):
  if torch.cuda.is_available():
    data, labels = data.cuda(), labels.cuda()

  target = custom_mobilenet(data.float())
  #print(target.shape)
  #print(labels.shape)

  # Getting accuracy
  predictions_ar.append(labels.cpu().detach().numpy())
  labels_ar.append(target.cpu().detach().numpy())   
  
predictions_ar = np.concatenate(predictions_ar)
labels_ar = np.concatenate(labels_ar)
accuracy = np.mean(labels_ar.argmax(axis=1)==predictions_ar)
print(f"Accuracy: {accuracy}")

In [ ]:
predictions_ar

In [ ]:
labels_ar.argmax(axis=1)